In [1]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re


if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/Desktop/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Methods.xlsx"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")


In [38]:
input = pd.read_excel(excel_path, sheet_name='Training Data')


In [39]:
# input['orig_source_population'] = input['Source Population'].apply(lambda x: ' '.join([word[0] for word in x.split()])).str.replace(r'\s', '')
input['pubmed_source_population'] = input['Source Population'].apply(lambda x: ''.join([word[0] for word in x.split()])).str.replace(r'&', '')
input = input.rename(columns={'Precision Medicine': 'orig_precision_medicine', 
                                  'Diabetes': 'orig_diabetes', 
                                  'Primary Study': 'orig_primary_study', 
                                  'Correct Source Population': 'orig_source_population'})

# Convert to lower
input['orig_precision_medicine'] = input['orig_precision_medicine'].str.lower()
input['orig_diabetes'] = input['orig_diabetes'].str.lower()
input['orig_primary_study'] = input['orig_primary_study'].str.lower()
input['orig_source_population'] = input['orig_source_population'].str.lower()
input['orig_source_population'] = input['orig_source_population'].fillna('unk')


input.drop(columns=['Source Population'], inplace=True)
input.head()




,PMID,Title,Abstract,MeSH,orig_precision_medicine,orig_diabetes,orig_source_population,orig_primary_study,pubmed_source_population
0,22744164,Acculturation and glycemic control of Asian In...,The prevalence of type 2 diabetes is dispropor...,Acculturation*; Asian / psychology; Asian / st...,no,yes,sa,yes,SA
1,15561964,Linkage analysis of diabetes status among hype...,Type 2 diabetes susceptibility is determined b...,"Chromosome Mapping*; Diabetes Mellitus, Type 2...",yes,yes,"na, ssa",yes,SA
2,28770629,"Ipragliflozin, a sodium glucose co-transporter...",Objective: We recently investigated the effect...,"Blood Glucose / analysis; Diabetes Mellitus, T...",no,yes,ea,yes,EA
3,33764184,Gastrodin protects against high glucose-induce...,Diabetic cardiomyopathy (DCM) is one of the ma...,Aryl Hydrocarbon Receptor Nuclear Translocator...,no,yes,unk,yes,EA
4,36155119,Curcumin supplementation reduces blood glucose...,Objective: To evaluate the effect of curcumin ...,Blood Glucose* / metabolism; Body Mass Index; ...,no,no,lac,yes,LAC


,PMID,Title,Abstract,MeSH,orig_precision_medicine,orig_diabetes,orig_source_population,orig_primary_study,pubmed_source_population,pmid,title,gpt_precision_medicine,gpt_diabetes,gpt_primary_study,gpt_source_population,source_population_match
0,22744164,Acculturation and glycemic control of Asian In...,The prevalence of type 2 diabetes is dispropor...,Acculturation*; Asian / psychology; Asian / st...,no,yes,sa,yes,SA,22744164,Acculturation and glycemic control of Asian I...,no,yes,yes,sa,True
1,15561964,Linkage analysis of diabetes status among hype...,Type 2 diabetes susceptibility is determined b...,"Chromosome Mapping*; Diabetes Mellitus, Type 2...",yes,yes,"na, ssa",yes,SA,15561964,Linkage analysis of diabetes status among hyp...,yes,yes,yes,na,True
2,28770629,"Ipragliflozin, a sodium glucose co-transporter...",Objective: We recently investigated the effect...,"Blood Glucose / analysis; Diabetes Mellitus, T...",no,yes,ea,yes,EA,28770629,"Ipragliflozin, a sodium glucose co-transporte...",no,yes,yes,ea,True
3,33764184,Gastrodin protects against high glucose-induce...,Diabetic cardiomyopathy (DCM) is one of the ma...,Aryl Hydrocarbon Receptor Nuclear Translocator...,no,yes,unk,yes,EA,33764184,Gastrodin protects against high glucose-induc...,yes,yes,yes,unk,True
4,36155119,Curcumin supplementation reduces blood glucose...,Objective: To evaluate the effect of curcumin ...,Blood Glucose* / metabolism; Body Mass Index; ...,no,no,lac,yes,LAC,36155119,Curcumin supplementation reduces blood glucos...,no,yes,yes,lac,True


In [118]:
""" results = pd.read_csv(path_equity_precision_llm_folder + '/llm training/Training_results.csv')

merged_df = input.merge(results, left_on='PMID', right_on='pmid', how='left')

import re

merged_df['source_population_match'] = merged_df.apply(lambda row: bool(re.search('(^|\s)' + row['gpt_source_population'],row['orig_source_population'])), axis=1)

# Crosstab summary for precision medicine
summary_precision_medicine = crosstab_summary(merged_df,truth='orig_precision_medicine',test='gpt_precision_medicine')
summary_diabetes = crosstab_summary(merged_df,truth='orig_diabetes',test='gpt_diabetes')
summary_primary_study = crosstab_summary(merged_df,truth='orig_primary_study',test='gpt_primary_study')

summary_precision_medicine['variable'] = 'Precision Medicine'
summary_diabetes['variable'] = 'Diabetes'
summary_primary_study['variable'] = 'Primary Study'  

t_source_population = pd.crosstab(merged_df['source_population_match'], merged_df['orig_source_population'])

prop_correct_source_population = t_source_population.loc[True].sum()/t_source_population.sum().sum() 
prop_correct_source_population

summary_source_population = pd.DataFrame({'variable': 'Source Population', 'Accuracy': prop_correct_source_population}, index=[0])


df_summary = pd.concat([summary_precision_medicine, summary_diabetes, summary_primary_study,summary_source_population])
df_summary['Scenario'] = 'Trial'
df_summary """


,Sensitivity,Specificity,Accuracy,ppv,npv,variable,Scenario
0,0.8,0.900000,0.85,0.888889,0.818182,Precision Medicine,Trial
0,1.0,0.444444,0.75,0.687500,1.000000,Diabetes,Trial
0,1.0,NaN,1.00,NaN,NaN,Primary Study,Trial
0,NaN,NaN,0.80,NaN,NaN,Source Population,Trial


### Running different scenarios (1 to 4)

In [130]:
import re
combined_output = pd.DataFrame()
for scenario in range(1,5):
    results = pd.read_csv(path_equity_precision_llm_folder + '/llm training/Training Scenario '+ str(scenario) +'_results.csv')
    merged_df = input.merge(results, left_on='PMID', right_on='pmid', how='left')
    merged_df['source_population_match'] = merged_df.apply(lambda row: bool(re.search('(^|\s)' + str(row['gpt_source_population']),str(row['orig_source_population']))), axis=1)

    # Crosstab summary for precision medicine
    summary_precision_medicine = crosstab_summary(merged_df,truth='orig_precision_medicine',test='gpt_precision_medicine')
    summary_diabetes = crosstab_summary(merged_df,truth='orig_diabetes',test='gpt_diabetes')
    summary_primary_study = crosstab_summary(merged_df,truth='orig_primary_study',test='gpt_primary_study')

    summary_precision_medicine['variable'] = 'Precision Medicine'
    summary_diabetes['variable'] = 'Diabetes'
    summary_primary_study['variable'] = 'Primary Study'  

    t_source_population = pd.crosstab(merged_df['source_population_match'], merged_df['orig_source_population'])

    prop_correct_source_population = t_source_population.loc[True].sum()/t_source_population.sum().sum() 
    prop_correct_source_population

    summary_source_population = pd.DataFrame({'variable': 'Source Population', 'Accuracy': prop_correct_source_population}, index=[0])


    df_summary = pd.concat([summary_precision_medicine, summary_diabetes, summary_primary_study,summary_source_population])
    df_summary['Scenario'] = 'Scenario' + str(scenario)
    combined_output = pd.concat([combined_output,df_summary],axis=0,ignore_index=True) 

combined_output.to_csv(path_equity_precision_llm_repo + '/preprocessing/epl03_combined output.csv', index=False)